In [3]:
import ROOT
import os
import numpy as np
from root_data_loader import load_data
import array

Welcome to JupyROOT 6.26/06


In [4]:
os.system('source /cms/ldap_home/yeonjoon/working_dir/VcbMVAStudy/setup.sh')
filepath = 'root://cluster142.knu.ac.kr//store/user/yeonjoon/Vcb_2018_Mu_Reco_Tree.root'
f = ROOT.TFile()
f = f.Open(filepath, 'READ')

In [5]:
tree = f.Get('Reco_45')
MVA_Score = np.array([0], dtype=np.float64)
tree.Branch("MVA_Score_template", MVA_Score, 'normal/D')

In [6]:
varlist = ['bvsc_w_u','bvsc_w_d','cvsl_w_u','cvsl_w_d','cvsb_w_u','cvsb_w_d','n_bjets','n_cjets','weight']
data = load_data(file_path=filepath,varlist=varlist,test_ratio=0,val_ratio=0,sigTree=['Reco_45'],bkgTree=['Reco_43'])
print(data['cat_columns'])

root://cluster142.knu.ac.kr//store/user/yeonjoon/Vcb_2018_Mu_Reco_Tree.root
Full dataset, For validation
[0.0053459  0.0043507  0.03749209 0.03283206 0.8746199  0.88254017
 0.         1.        ]
['n_bjets', 'n_cjets']


In [7]:
from pytorch_tabnet.tab_model import TabNetClassifier
print(data['cat_dims'])
model = TabNetClassifier()
model.load_model('/cms/ldap_home/yeonjoon/working_dir/VcbMVAStudy/TabNet_template/model_largebatch_nosmote.pt.zip')


/cms/ldap_home/yeonjoon/miniconda3/envs/ML-torch/lib/python3.10/site-packages/ROOT/_facade.py:153: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  return _orig_ihook(name, *args, **kwds)


[7, 12]


/cms/ldap_home/yeonjoon/miniconda3/envs/ML-torch/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [17]:
from pytorch_tabnet.tab_model import TabNetClassifier
print(data['cat_dims'])
model = TabNetClassifier()
model.load_model('/cms/ldap_home/yeonjoon/working_dir/VcbMVAStudy/TabNet_template/model_largebatch_nosmote.pt.zip')
def getMVA(bvsc_w_u,bvsc_w_d,cvsl_w_u,cvsl_w_d,cvsb_w_u,cvsb_w_d,n_bjets,n_cjets):

    train_features=[bvsc_w_u,
                    bvsc_w_d,
                    cvsl_w_u,
                    cvsl_w_d,
                    cvsb_w_u,
                    cvsb_w_d,
                    data['cat_labelencoder']['n_bjets'].transform([n_bjets])[0],
                    data['cat_labelencoder']['n_cjets'].transform([n_cjets])[0]]
    train_features = np.array(train_features)
    
    return model.predict_proba([train_features])[0,1]

[7, 12]


/cms/ldap_home/yeonjoon/miniconda3/envs/ML-torch/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [20]:
f_out = ROOT.TFile("out.root","RECREATE")    

In [21]:
tree_list = ['Reco_45','Reco_43','Reco_41','Reco_23','Reco_21']

In [22]:
import tqdm
from array import array
for name in tree_list:
    tree = f.Get(name)
    tree_out = ROOT.TTree(name,name)
    MVAs = array('d', [0])
    weights = array('d', [0])
    tree_out.Branch("MVA",MVAs,"MVA/D")
    tree_out.Branch("weights",weights,"weights/D")

    for i, entry in enumerate(tqdm.tqdm(tree)):
        weights[0] = float(entry.weight)
        MVAs[0] = float(getMVA(entry.bvsc_w_u,entry.bvsc_w_d,entry.cvsl_w_u,entry.cvsl_w_d,entry.cvsb_w_u,entry.cvsb_w_d,entry.n_bjets,entry.n_cjets))
        tree_out.Fill()
    f_out.cd()
    tree_out.write()

23521it [07:09, 54.72it/s]


KeyboardInterrupt: 

In [11]:
for name in tree_list:
    data = load_data(file_path=filepath,varlist=varlist,test_ratio=0,val_ratio=0,sigTree=['Reco_45'],bkgTree=[])
    tree = f.Get(name)
    print(tree.GetEntries())
    print(data['train_features'].shape)
    tree_out = ROOT.TTree(name,name)
    MVAs = array('d', [0])
    weights = array('d', [0])
    tree_out.Branch("MVA",MVAs,"MVA/D")
    tree_out.Branch("weights",weights,"weights/D")
    result = model.predict_proba(data['train_features'])[:,1]
    
    for i, entry in enumerate(result):
        weights[0] = float(result[i])
        MVAs[0] = float(data['train_weight'][i])
        tree_out.Fill()
    f_out.cd()
    tree_out.write()

******************************************************************************
*Tree    :Reco_45   : Reco_45                                                *
*Entries :   164280 : Total =        15169530 bytes  File  Size =   10952940 *
*        :          : Tree compression factor =   1.38                       *
******************************************************************************
*Br    0 :weight    : weight/F                                               *
*Entries :   164280 : Total  Size=     659445 bytes  File Size  =     572561 *
*Baskets :       21 : Basket Size=      32000 bytes  Compression=   1.15     *
*............................................................................*
*Br    1 :n_jets    : n_jets/I                                               *
*Entries :   164280 : Total  Size=     659445 bytes  File Size  =     105347 *
*Baskets :       21 : Basket Size=      32000 bytes  Compression=   6.25     *
*...................................................